# RNNs, LSTMs, and the Capital Markets

# Key Takeaways:
* This is an exploration of the performance of recurrent neural networks and long-short term memory nodes on data from the U.S. capital markets.

* Data were limited to monthly frequency and could easily be expanded to alternate timesteps and augmented with economic data.

* The purpose was primarily to understand the coding of the network and not create an engine for trading decisions (don't use this for trading).

* After tinkering with hyperparameters, we concluded that simpler models (i.e., fewer hidden layers) may produce better results, as simple models produced results consistent with our statistical intuition that this is a random walk, but more data could change this.

* The activation function turned out to be a fascinating aspect of the model with hyperbolic tangent, 'tanh', potentially having a simplfying affect on the network not exhibited by 'softmax' or 'reLU.'

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import StandardScaler

# 1.0 Data Ingestion

We dowloaded monthly market data for the S&P SPY exchange traded fund, as only intsitutions can truly buy a 500 stock approximation to the U.S. markets. We opted not to hook this up to the yfinance() API, as unexpected changes in the API could render the notebook useless.

In [23]:
# Load data
data = pd.read_csv('~/Desktop/Springboard/Capstone3/marketdata2.csv')
data.head()


,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-02-01,43.96875,45.12500,42.81250,44.40625,25.487274,5417600
1,1993-03-01,44.56250,45.84375,44.21875,45.18750,25.935671,3019200
2,1993-04-01,45.25000,45.25000,43.28125,44.03125,25.391386,2697200
3,1993-05-01,44.09375,45.65625,43.84375,45.21875,26.076170,1808000
4,1993-06-01,45.37500,45.81250,44.21875,45.06250,25.986073,3438000


The raw data could be used as is, however, we're going to add value via computation and a bit of industry knowledge.

# 2.0 Data Cleaning 

It's always a good idea to view your data from 35,000 feet. 

In [24]:
data.shape

(362, 7)

Checking the types of data one has been handed is simple common sense: If the encoding is wrong, you'll likely get more errors than meaningful results.

In [25]:
data.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

The raw data are in great shape. Only the Date column wants for typecasting.

In [26]:
data['Date'] = pd.to_datetime(data['Date'])

# 2.0 Data Wrangling

According to the output, we have 362 monthly observations and 7 features, all of which are intuitively labeled based on what they tell us about SPY that day.

Percentage change is a closely watched metric in finance, but quite a number of percentages can be calculated from the data. Below, we calculated every reasonable percentage change from the raw data. This doesn't tell us as one might think, given that industry insiders compare daily, weekly, monthly, and annual percentages to guage the current state of the market. They do not weight them equally, rather this tradeoff is partially based on an endless parade of economic data and, sadly, some psychological bias.

In [27]:
# https://stackoverflow.com/questions/3637781
data['pct_chg_open'] = (-data['Open'].diff())/data['Open'].shift()
data['pct_chg_close'] = (-data['Close'].diff())/data['Close'].shift()
data['pct_chg_high'] = (-data['High'].diff())/data['High'].shift()
data['pct_chg_low'] = (-data['Low'].diff())/data['Low'].shift()
data['pct_chg_vol'] = (-data['Volume'].diff())/data['Volume'].shift()
data['pct_chg_high_open'] = data['High']/data['Open']-1
data['pct_chg_open_high'] = data['Open']/data['High']-1
data['pct_chg_open_low'] = data['Open']/data['Low']-1
data['pct_chg_low_open'] = data['Low']/data['Open']-1
data['pct_chg_high_low'] = data['High']/data['Low']-1
data['pct_chg_low_high'] = data['Low']/data['High']-1
data['pct_chg_close_open'] = data['Close']/data['Open']-1
data['pct_chg_open_close'] = data['Open']/data['Close']-1

In [28]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,pct_chg_open,pct_chg_close,pct_chg_high,pct_chg_low,pct_chg_vol,pct_chg_high_open,pct_chg_open_high,pct_chg_open_low,pct_chg_low_open,pct_chg_high_low,pct_chg_low_high,pct_chg_close_open,pct_chg_open_close
0,1993-02-01,43.96875,45.12500,42.81250,44.40625,25.487274,5417600,NaN,NaN,NaN,NaN,NaN,0.026297,-0.025623,0.027007,-0.026297,0.054015,-0.051247,0.009950,-0.009852
1,1993-03-01,44.56250,45.84375,44.21875,45.18750,25.935671,3019200,-0.013504,-0.017593,-0.015928,-0.032847,0.442705,0.028752,-0.027948,0.007774,-0.007714,0.036749,-0.035446,0.014025,-0.013831
2,1993-04-01,45.25000,45.25000,43.28125,44.03125,25.391386,2697200,-0.015428,0.025588,0.012952,0.021201,0.106651,0.000000,0.000000,0.045487,-0.043508,0.045487,-0.043508,-0.026934,0.027679
3,1993-05-01,44.09375,45.65625,43.84375,45.21875,26.076170,1808000,0.025552,-0.026969,-0.008978,-0.012996,0.329675,0.035436,-0.034223,0.005702,-0.005670,0.041340,-0.039699,0.025514,-0.024879
4,1993-06-01,45.37500,45.81250,44.21875,45.06250,25.986073,3438000,-0.029057,0.003455,-0.003422,-0.008553,-0.901549,0.009642,-0.009550,0.026148,-0.025482,0.036042,-0.034789,-0.006887,0.006935


Percentage changes are important, but guaging the trend of multiple, non-stationary time series is generally done with moving averages, the most popular of which are calculated below. 

In [29]:
data['ma30'] = data['Close'].rolling(30).mean()
data['ma50'] = data['Close'].rolling(50).mean()
data['ma90'] = data['Close'].rolling(90).mean()
data['ma100'] = data['Close'].rolling(100).mean()
data['ma200'] = data['Close'].rolling(200).mean()

Moving averages aren't used in isolation, but rather compared to the price. So, if the price is above the 50 or 200 day moving averages, the stock is overbought and investors are likely to let it decline, until the price approximates the 30 day moving average, at which point investors will be drawn to the stock that is "on sale" (i.e., oversold). Such investors, however, would do well to remember there are two types of stock in the world: stocks that are cheap and stocks that are cheap for a reason. This approach to investing neglects potential reasons stocks may be selling at discount prices.

# 3.0 Feature Engineering

We'll add some additional features to our model, though, theoretically we could find hundreds of features to use.

First, we'll calculate the Relative Strength Index (RSI). Traders use RSI as one factor when deciding if a stock is cheap (i.e., "oversold") or expensive (i.e., "overbought"). 
Second, we'll interpret the moving averages discussed above and determine whether or not they are sending allegedly bullish or bearish signals.

In [30]:
# https://en.wikipedia.org/wiki/Relative_strength_index
# https://www.qmr.ai/relative-strength-index-rsi-in-python/

# calculate percentages 
change = data["Close"].diff()

change_up = change.copy()
change_down = change.copy()

change_up[change_up<0] = 0
change_down[change_down>0] = 0

# Error Check
change.equals(change_up+change_down)

# Calculate the rolling average of average up and average down
avg_up = change_up.rolling(14).mean()
avg_down = change_down.rolling(14).mean().abs()

data['rsi'] = 100 * avg_up / (avg_up + avg_down)

# Interpret signal
data['RSI_signal_down'] = data['rsi'].apply(lambda x: 1 if x < 30 else 0)
data['RSI_signal_up'] = data['rsi'].apply(lambda x: 0 if x < 70 else 1)

# View 20 oldest datapoints
data.head(20)

,Date,Open,High,Low,Close,Adj Close,Volume,pct_chg_open,pct_chg_close,pct_chg_high,...,pct_chg_close_open,pct_chg_open_close,ma30,ma50,ma90,ma100,ma200,rsi,RSI_signal_down,RSI_signal_up
0,1993-02-01,43.968750,45.125000,42.812500,44.406250,25.487274,5417600,NaN,NaN,NaN,...,0.009950,-0.009852,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,1993-03-01,44.562500,45.843750,44.218750,45.187500,25.935671,3019200,-0.013504,-0.017593,-0.015928,...,0.014025,-0.013831,NaN,NaN,NaN,NaN,NaN,NaN,0,1
2,1993-04-01,45.250000,45.250000,43.281250,44.031250,25.391386,2697200,-0.015428,0.025588,0.012952,...,-0.026934,0.027679,NaN,NaN,NaN,NaN,NaN,NaN,0,1
3,1993-05-01,44.093750,45.656250,43.843750,45.218750,26.076170,1808000,0.025552,-0.026969,-0.008978,...,0.025514,-0.024879,NaN,NaN,NaN,NaN,NaN,NaN,0,1
4,1993-06-01,45.375000,45.812500,44.218750,45.062500,25.986073,3438000,-0.029057,0.003455,-0.003422,...,-0.006887,0.006935,NaN,NaN,NaN,NaN,NaN,NaN,0,1
5,1993-07-01,45.125000,45.218750,44.156250,44.843750,26.043200,6117600,0.005510,0.004854,0.012960,...,-0.006233,0.006272,NaN,NaN,NaN,NaN,NaN,NaN,0,1
6,1993-08-01,44.906250,46.562500,44.843750,46.562500,27.041382,5440100,0.004848,-0.038328,-0.029717,...,0.036882,-0.035570,NaN,NaN,NaN,NaN,NaN,NaN,0,1
7,1993-09-01,46.406250,46.593750,44.812500,45.937500,26.678417,4369900,-0.033403,0.013423,-0.000671,...,-0.010101,0.010204,NaN,NaN,NaN,NaN,NaN,NaN,0,1
8,1993-10-01,45.875000,47.156250,45.718750,46.843750,27.374218,6972900,0.011448,-0.019728,-0.012072,...,0.021117,-0.020680,NaN,NaN,NaN,NaN,NaN,NaN,0,1
9,1993-11-01,46.781250,47.000000,45.531250,46.343750,27.082035,5351100,-0.019755,0.010674,0.003313,...,-0.009352,0.009440,NaN,NaN,NaN,NaN,NaN,NaN,0,1


The RSI calculation above is popular. Building off of what was already available in on the web made our slightly less transparent calculation significantly better (see above citations). We now take on the meanings of percentage/price comparisons.

In [31]:
# compare daily closing prices to moving averages
data['above30'] = data['Close'] > data['ma30']
data['above50'] = data['Close'] > data['ma50']
data['above90'] = data['Close'] > data['ma90']
data['above100'] = data['Close'] > data['ma100']
data['above200'] = data['Close'] > data['ma200']

# make sure we have the data types we expect
data['above30'] = data['above30'].astype(int)
data['above50'] = data['above50'].astype(int)
data['above90'] = data['above90'].astype(int)
data['above100'] = data['above100'].astype(int)
data['above200'] = data['above200'].astype(int)

In [32]:
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,pct_chg_open,pct_chg_close,pct_chg_high,...,ma100,ma200,rsi,RSI_signal_down,RSI_signal_up,above30,above50,above90,above100,above200
357,2022-11-01,390.140015,407.679993,368.790009,407.679993,405.816467,1745985300,-0.080481,-0.055592,-0.046621,...,292.290499,213.336649,46.676368,0,0,1,1,1,1,1
358,2022-12-01,408.769989,410.489990,374.769989,382.429993,380.681885,1735926500,-0.047752,0.061936,-0.006893,...,294.107699,214.591449,37.920622,0,0,0,1,1,1,1
359,2023-01-01,384.369995,408.160004,377.829987,406.480011,406.480011,1574934300,0.059691,-0.062887,0.005676,...,296.202299,215.986299,42.746943,0,0,1,1,1,1,1
360,2023-02-01,405.209991,418.309998,398.820007,399.089996,399.089996,1137807500,-0.054219,0.018181,-0.024868,...,298.176599,217.345349,38.375266,0,0,0,1,1,1,1
361,2023-02-21,403.059998,404.160004,398.820007,399.089996,399.089996,82655924,0.005306,-0.000000,0.033827,...,300.095499,218.701549,41.565985,0,0,0,1,1,1,1


We know our data are monthly and when the next prediction hits, so we don't need the last row. At the time of this writing, tomorrow's close will provide evidence of whether or not this model requires an additional hundred or more predictors. Rows with `na's` also have no probative value. We take care of this last minute clean-up in the next cell.

In [33]:
dates = data['Date']
data = data.iloc[:-1 , :]
data = data.dropna()

With a data shape like the one below, we shouldn't suffer from rank deficiency. 

In [34]:
data.shape

(162, 33)

In [35]:
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,pct_chg_open,pct_chg_close,pct_chg_high,...,ma100,ma200,rsi,RSI_signal_down,RSI_signal_up,above30,above50,above90,above100,above200
356,2022-10-01,361.079987,389.519989,348.109985,386.209991,384.444611,2024732000,0.080964,-0.081276,0.053943,...,290.144599,211.947399,39.908580,0,0,0,1,1,1,1
357,2022-11-01,390.140015,407.679993,368.790009,407.679993,405.816467,1745985300,-0.080481,-0.055592,-0.046621,...,292.290499,213.336649,46.676368,0,0,1,1,1,1,1
358,2022-12-01,408.769989,410.489990,374.769989,382.429993,380.681885,1735926500,-0.047752,0.061936,-0.006893,...,294.107699,214.591449,37.920622,0,0,0,1,1,1,1
359,2023-01-01,384.369995,408.160004,377.829987,406.480011,406.480011,1574934300,0.059691,-0.062887,0.005676,...,296.202299,215.986299,42.746943,0,0,1,1,1,1,1
360,2023-02-01,405.209991,418.309998,398.820007,399.089996,399.089996,1137807500,-0.054219,0.018181,-0.024868,...,298.176599,217.345349,38.375266,0,0,0,1,1,1,1


Before we build our model, we must standardize large values that could dominate our analysis.

In [36]:
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'pct_chg_open', 'pct_chg_close', 'pct_chg_high', 'pct_chg_low',
       'pct_chg_vol', 'pct_chg_high_open', 'pct_chg_open_high',
       'pct_chg_open_low', 'pct_chg_low_open', 'pct_chg_high_low',
       'pct_chg_low_high', 'pct_chg_close_open', 'pct_chg_open_close', 'ma30',
       'ma50', 'ma90', 'ma100', 'ma200', 'rsi', 'RSI_signal_down',
       'RSI_signal_up', 'above30', 'above50', 'above90', 'above100',
       'above200'],
      dtype='object')

In [40]:
cols_to_standardize = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'pct_chg_open', 'pct_chg_close', 'pct_chg_high', 'pct_chg_open_low', 'pct_chg_low_open', 'pct_chg_high_low',
       'pct_chg_low_high', 'pct_chg_close_open', 'pct_chg_open_close', 'ma30',
       'ma50', 'ma90', 'ma100', 'ma200', 'rsi']

# instantiate scaler
scaler = StandardScaler()

# scale and transform columns
scaler.fit(data[cols_to_standardize])
data[cols_to_standardize] = scaler.transform(data[cols_to_standardize])

print(data.head()

          Date      Open      High       Low     Close  Adj Close    Volume  \
199 2009-09-01 -1.350885 -1.320469 -1.356953 -1.330954  -1.282804  1.031806   
200 2009-10-01 -1.340429 -1.298887 -1.331622 -1.351209  -1.293927  1.598059   
201 2009-11-01 -1.329177 -1.285172 -1.320213 -1.287550  -1.247528  0.752527   
202 2009-12-01 -1.261563 -1.272798 -1.258036 -1.272584  -1.236619  0.312410   
203 2010-01-01 -1.247125 -1.252560 -1.276877 -1.312994  -1.261868  0.974585   

     pct_chg_open  pct_chg_close  pct_chg_high  ...     ma100     ma200  \
199     -0.253969      -0.496397     -1.014865  ... -1.012894 -1.300714   
200     -0.013564       0.668829     -0.452831  ... -1.016406 -1.291324   
201     -0.028609      -1.223487     -0.153026  ... -1.018511 -1.281046   
202     -1.243498      -0.100651     -0.098896  ... -1.019010 -1.270346   
203     -0.075687       1.069550     -0.370647  ... -1.018466 -1.260478   

          rsi  RSI_signal_down  RSI_signal_up  above30  above50  above90  

In [43]:
data.drop(['Date'], axis=1, inplace=True)

# 4.0 Model Training

We're ready to make some magic happen or, at least, build our recurrent neural network. We begin by converting our data to an array. Since we'll be in array form, we won't use the typical train/test split syntax, but rather split our data manually.

In [44]:
values = data.values
train_size = int(len(values) * 0.7)
train = values[0:train_size,:] 
test = values[train_size:len(values),:]

We're now ready to define our model and hyperparameters. Note, our target is in column 0.

In [57]:
data.shape

(162, 32)

In [87]:
# inspired by MIT.S.191 -- intro to deep learning

# Define function to create LSTM model
def create_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(1, look_back), activation='tanh'))
    model.add(Dense(20, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# let's look back one month
look_back = 1

# these are boilerplate values
epochs = 100
batch_size = 1

trainX, trainY = [], []
testX, testY = [], []
for i in range(look_back, len(train)):
    trainX.append(train[i-look_back:i, 0])
    trainY.append(train[i, 0])
for i in range(look_back, len(test)):
    testX.append(test[i-look_back:i, 0])
    testY.append(test[i, 0])

# Fit to LSTM input format
trainX = np.reshape(trainX, (len(trainX), 1, look_back))
testX = np.reshape(testX, (len(testX), 1, look_back))
trainY = np.array(trainY)
testY = np.array(testY)


model = create_model()
model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=2)

# Make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Print model performance metrics
trainScore = model.evaluate(trainX, trainY, verbose=0)
testScore = model.evaluate(testX, testY, verbose=0)
print('Train Score: %.2f MSE' % (trainScore))
print('Test Score: %.2f MSE' % (testScore))

Epoch 1/100
112/112 - 1s - loss: 0.1877 - 877ms/epoch - 8ms/step
Epoch 2/100
112/112 - 0s - loss: 0.0518 - 101ms/epoch - 905us/step
Epoch 3/100
112/112 - 0s - loss: 0.0326 - 106ms/epoch - 943us/step
Epoch 4/100
112/112 - 0s - loss: 0.0292 - 100ms/epoch - 894us/step
Epoch 5/100
112/112 - 0s - loss: 0.0308 - 103ms/epoch - 921us/step
Epoch 6/100
112/112 - 0s - loss: 0.0249 - 97ms/epoch - 866us/step
Epoch 7/100
112/112 - 0s - loss: 0.0277 - 94ms/epoch - 838us/step
Epoch 8/100
112/112 - 0s - loss: 0.0253 - 94ms/epoch - 842us/step
Epoch 9/100
112/112 - 0s - loss: 0.0267 - 93ms/epoch - 834us/step
Epoch 10/100
112/112 - 0s - loss: 0.0222 - 99ms/epoch - 882us/step
Epoch 11/100
112/112 - 0s - loss: 0.0258 - 100ms/epoch - 893us/step
Epoch 12/100
112/112 - 0s - loss: 0.0228 - 100ms/epoch - 890us/step
Epoch 13/100
112/112 - 0s - loss: 0.0225 - 104ms/epoch - 925us/step
Epoch 14/100
112/112 - 0s - loss: 0.0234 - 107ms/epoch - 953us/step
Epoch 15/100
112/112 - 0s - loss: 0.0223 - 101ms/epoch - 906us/s

# 5.0 Model Evaluation

In [56]:
# Train and Predict LSTM model
model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=2)

trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Print model performance metrics
trainScore = model.evaluate(trainX, trainY, verbose=0)
testScore = model.evaluate(testX, testY, verbose=0)
print('Train Score: %.2f MSE' % (trainScore))
print('Test Score: %.2f MSE' % (testScore))

Epoch 1/100
112/112 - 1s - loss: 0.3693 - 846ms/epoch - 8ms/step
Epoch 2/100
112/112 - 0s - loss: 0.0994 - 88ms/epoch - 786us/step
Epoch 3/100
112/112 - 0s - loss: 0.0461 - 91ms/epoch - 813us/step
Epoch 4/100
112/112 - 0s - loss: 0.0216 - 91ms/epoch - 815us/step
Epoch 5/100
112/112 - 0s - loss: 0.0097 - 88ms/epoch - 790us/step
Epoch 6/100
112/112 - 0s - loss: 0.0061 - 88ms/epoch - 783us/step
Epoch 7/100
112/112 - 0s - loss: 0.0054 - 88ms/epoch - 782us/step
Epoch 8/100
112/112 - 0s - loss: 0.0052 - 86ms/epoch - 771us/step
Epoch 9/100
112/112 - 0s - loss: 0.0051 - 89ms/epoch - 797us/step
Epoch 10/100
112/112 - 0s - loss: 0.0052 - 88ms/epoch - 786us/step
Epoch 11/100
112/112 - 0s - loss: 0.0050 - 89ms/epoch - 797us/step
Epoch 12/100
112/112 - 0s - loss: 0.0050 - 91ms/epoch - 811us/step
Epoch 13/100
112/112 - 0s - loss: 0.0049 - 90ms/epoch - 808us/step
Epoch 14/100
112/112 - 0s - loss: 0.0048 - 92ms/epoch - 823us/step
Epoch 15/100
112/112 - 0s - loss: 0.0049 - 92ms/epoch - 817us/step
Epoch

# 6.0 Implications & Conclusion

We don't think anyone is going to get rich using this model. The training score essentially says our model memorized our data. Our test score is, as always, more interesting. The test score MSE increases with the number of periods our model looks back before making a decision. At a lookback of 4 months, our test MSE was an uninspiring .54. Yet, looking back 1 month yielded a test MSE of ~.20. This seems to lend credence to the view that the best point estimate of a random walk is the value from a comparable prior period. In our experience with this model, the opening price this month really was best predicted by the price the month prior. We can think of no reality in which this would change by using the closing value as a target.

One should note that adding complexity only worsened the problem. Adding a 20 node dense layer pushed the test MSE back to ~.5 with a 'relu' activation function, but dropped back to .20 when the activation function was changed to 'softmax.' We do not believe model selection should be driven by the activation function, so we were unimpressed by this unexpected result. Removing the 20 intermediate, hidden layer and adding 20% dropout brought our test MSE back to .20 with 'relu' activation and 'softmax.' The only exception to our "simpler is better" contention occurred when we switched the activation function to 'tanh.' Hyperbolic tangent ('tanh') prevents gradient explosion and has output values of -1, 0, and 1. It's value is derived from the interplay of odd and even values defined by an exponential function. This restriction may simplify output such that the complexity of the model's network in masked. This question should be addressed in future research, as should some attempt at standardizing the activation functions chosen for different problems.